In [2]:
!pip install opencv-python
import cv2
import numpy as np
import os
import math

img1_path = "/content/02 Sampling Grevys_Zebra_Stallion.jpg"
img2_path = "/content/02 Sampling Trevarno_pavo_cristatus06.jpg"

img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

if img1 is None or img2 is None:
    raise FileNotFoundError("One of the image paths is incorrect!")

# Convert to grayscale
g1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
g2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)


def freq_sample(gray, keep_frac):
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    rows, cols = gray.shape

    kr = max(1, int(rows * keep_frac))
    kc = max(1, int(cols * keep_frac))

    mask = np.zeros_like(fshift)
    r0 = rows // 2 - kr // 2
    c0 = cols // 2 - kc // 2

    mask[r0:r0 + kr, c0:c0 + kc] = 1
    filtered = fshift * mask

    back = np.fft.ifft2(np.fft.ifftshift(filtered))
    back = np.real(back)

    back -= back.min()
    if back.max() > 0:
        back = back / back.max() * 255

    return back.astype(np.uint8)


def spatial_sample(gray, factor):
    r, c = gray.shape
    down = cv2.resize(gray, (c // factor, r // factor), interpolation=cv2.INTER_AREA)
    up = cv2.resize(down, (c, r), interpolation=cv2.INTER_LINEAR)
    return up


output_dir = "sampling_outputs"
os.makedirs(output_dir, exist_ok=True)


factors = [2, 4, 8, 16]

for name, gray in [("zebra", g1), ("peacock", g2)]:
    for f in factors:

        freq_img = freq_sample(gray, 1 / f)
        cv2.imwrite(f"{output_dir}/{name}_frequency_1over{f}.png", freq_img)


        spatial_img = spatial_sample(gray, f)
        cv2.imwrite(f"{output_dir}/{name}_spatial_1over{f}.png", spatial_img)

print("Sampling complete")

Sampling complete
